<div style="max-width:1200px"><img src="../_resources/mgnify_banner.png" width="100%"></div>

<img src="../_resources/mgnify_logo.png" width="200px">

# Comparative metagenomics at community-level

## Normalization methods and alpha & beta diversity

[MGnifyR](http://github.com/beadyallen/mgnifyr) is a library that provides a set of tools for easily accessing and processing MGnify data in R, making queries to MGnify databases through the [MGnify API](https://www.ebi.ac.uk/metagenomics/api/v1/). 
The benefit of MGnifyR is that data can either be fetched in tsv format or be directly combined in a phyloseq object to run an analysis in a custom workflow.

In this example we aim to demonstrate how the MGnifyR tool can be used to fetch data and metadata of a MGnify metagenomic analyisis. Then we show how to generate diversity metrics for comparative metagenomics using taxonomic profiles.

This is an interactive code notebook (a Jupyter Notebook). To run this code, click into each cell and press the ▶ button in the top toolbar, or press shift+enter

## Contents
- [Part 1. Fetch data from MGnify using MGnifyR, explore metadata and build a phyloseq object](#part1)
  - [1.1. Fetch the MGnify Analyses accession](#part1_1)
  - [1.2. Explore and filter samples by metadata](#part1_2)
  - [1.3. Converting into phyloseq object](#part1_3)
- [Part 2. Normalization, alpha diversity indices and taxonomic profiles visualization](#part2)
   - [2.1. Cleaning the OTUs matrix](#part2_1)
   - [2.2. Normalization by total sum scaling (TSS, relative abundance or proportions)](#part2_2)
   - [2.3. Normalization by subsampling (rarefaction)](#part2_3)
   - [2.4. Normalization by cumulative sum scaling (CSS)](#part2_4)
- [Part 3. Comparative metagenomics at community-level: Beta diversity](#part3)
- [References](#refs)

In [ ]:
library(IRdisplay)
#display_markdown(file = '../_resources/mgnifyr_help.md')

Loading libraries:

In [ ]:
library(stringr)
library(vegan)
library(ggplot2)
library(phyloseq)
library(metagenomeSeq)
library(MGnifyR)
library(microbiomeMarker)
library(plyr)

mg = mgnify_client(usecache = T, cache_dir = '/home/jovyan/.mgnify_cache')

Setting tables and figures size to display (these will be reset later):

In [ ]:
options(repr.matrix.max.cols=150, repr.matrix.max.rows=200)
options(repr.plot.width=6, repr.plot.height=6)

## Part 1. Fetch data from MGnify using MGnifyR, explore metadata and build a phyloseq object <a id='part1'/>

In this example we are going to fetch MGnify analysis results and metadata for TARA ocean metagenomic study corresponding to size fractions for prokaryotes ([MGYS00002008](https://www.ebi.ac.uk/metagenomics/studies/MGYS00002008#overview)).
Find more information about the [TARA Ocean Project.](https://fondationtaraocean.org/en/expedition/tara-oceans/)

### 1.1. Fetch the MGnify Analyses accession <a id='part1_1'/>

The first step is to retrieve the analysis accession list.

In [ ]:
tara_all = mgnify_analyses_from_studies(mg, 'MGYS00002008')

And use this list to fetch the metadata for all of the analyses from the MGnify API.

In [ ]:
metadata = mgnify_get_analyses_metadata(mg, tara_all)
#head(metadata)

Note: In case you are intereseted in running the comparative metagenomic analysis using data from different studies in MGnify, you can adapt the following commands:

```R
analyses_accessions = mgnify_analyses_from_studies(mg, c("MGYS1","MGYS2"))

metadata = mgnify_get_analyses_metadata(mg, analyses_accessions)
```

### 1.2. Explore and filter samples by metadata <a id='part1_2'/>

We want to keep only **metagenomic** samples (noy amplicon) of 'surface water layer ([ENVO:00002042](https://www.ebi.ac.uk/ols/ontologies/envo/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FENVO_00002042))' and 'mesopelagic zone ([ENVO:00000213](https://www.ebi.ac.uk/ols/ontologies/envo/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FENVO_00000213))' to compare. We also want to filter out results generated with old pipeline versions (<[V5.0](https://www.ebi.ac.uk/metagenomics/pipelines/5.0)). In the following steps we will filter out other samples before exporting to the phyloseq object. Let's first explore the metadata we fetched:

1) Check the number of analysis in the study.

In [ ]:
length(metadata$'analysis_accession')

2) Check the `analysis_experiment-type` to determine whether a filtering is necesary to discard amplicon samples.

In [ ]:
unique(metadata$'analysis_experiment-type')

3) Keep results generated only with the most updated pipeline (v5.0).

In [ ]:
v5_metadata = metadata[which(metadata$'analysis_pipeline-version'=='5.0'), ]
length(v5_metadata$'analysis_accession')
#head(v5_metadata)

4) Check the `sample_environment-feature` to discover what kind of samples are part of the study and how many of each exists.

*Note that for a comparative study, we need at least 5 samples per group.*

In [ ]:
table(v5_metadata$'sample_environment-feature')

Let's keep only samples having [ENVO:00002042](https://www.ebi.ac.uk/ols/ontologies/envo/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FENVO_00002042) or [ENVO:00000213](https://www.ebi.ac.uk/ols/ontologies/envo/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FENVO_00000213) in the `sample_environment-feature` column. We want to create a new dataframe containing the relevant samples only.

We are also going to create a clean label for the environment feature.

<div class="alert alert-block alert-info">
To speed up the following analysis, we are going to keep only 25 samples per group (by randomly subsampling the accessions)
</div>

In [ ]:
# To create a dataframe with the relevant samples
sub1 = v5_metadata[str_detect(v5_metadata$'sample_environment-feature', "ENVO:00002042"), ]
set.seed(345)
acc_s1=sample(sub1$'analysis_accession', size=25, replace = FALSE)

sub2 = v5_metadata[str_detect(v5_metadata$'sample_environment-feature', "ENVO:00000213"), ]
set.seed(345)
acc_s2=sample(sub2$'analysis_accession', size=25, replace = FALSE)

filtered_samples = c(acc_s1,acc_s2)

# To create the label:
env_label=c(rep('Surface', times=25), rep('Mesopelagic', times=25))


### 1.3. Converting into phyloseq object <a id='part1_3'/>

Now that we have a new dataframe with 162 samples from either surface or mesopelagic zone water, we are going to create a phyloseq object label it with the environment feature.

In [ ]:
ps = mgnify_get_analyses_phyloseq(mg, filtered_samples)
sample_data(ps)$'env_feature' = env_label

## Part 2. Normalization, alpha diversity indices and taxonomic profiles visualization <a id='part2'/>

### 2.1. Cleaning the OTUs matrix <a id='part2_1'/>

1) Remove samples with extremely low coverage – they aren't informative and interfere with the normalization process. The first step is to detect outliers by plotting some histograms.

In [ ]:
hist(log10(sample_sums(ps)), breaks = 50)

We can see that samples with number of reads $\leq 10 ^ {1.5}$ (i.e. $\lesssim 32$) seem to be outliers. 
   Let's filter out the outliers and plot a new histogram.

In [ ]:
ps_good = subset_samples(ps, sample_sums(ps) > 32)
ps_good
hist(log10(sample_sums(ps_good)), breaks = 50)

And let's check how many samples were discarded:

In [ ]:
nsamples(ps)
nsamples(ps_good)

2) Remove singletons existing in a single sample. Singletons are OTUs of size one, meaning that only one read was assigned to that OTU. These very low-abundance OTUs could be biologically real (belonging to the rare biosphere [1]), or they could be false positives due to sequencing artefacts. Singletons observed in only one sample are more likely to be artefacts, and it is good practice to remove them from the OTUs counts table to avoid artificially over-estimating the OTUs richness. You can find more discusssion about this in [Robert Edgar's blog](https://drive5.com/usearch/manual/singletons.html).

In [ ]:
ps_final <- filter_taxa(ps_good, function(x) sum(x) > 1, prune=TRUE)
ps_final

3) Show some stats on the sequencing depth across samples.

In [ ]:
max_difference=max(sample_sums(ps_final))/min(sample_sums(ps_final))

sprintf("The max difference in sequencing depth is %s", max_difference)

boxplot(sample_sums(ps_final))
text(y = boxplot.stats(sample_sums(ps_final))$stats, labels = boxplot.stats(sample_sums(ps_final))$stats, x = 1.25)

An approximately 10-fold difference in the library sizes means that we will need to apply a normalization method before continuing with the analysis. The most common normalization methods used in microbiome count data are proportions and rarefaction. However, other methods originally developed to normalize RNA-seq counts have been adapted to differential-abundance analysis in microbiome data. A discussion about how to choose the right normalization method is out of the scope of this material, but the topic has been covered in multiple forums and scientific publications. Depending on the downstream analysis we intend to perform, different methods might be appropriate. For instance, to compare at community-level through beta-diversity, "...proportions and rarefying produced more accurate comparisons among communities and are the only methods that fully normalized read depths across samples. Additionally, upper quartile, cumulative sum scaling (CSS), edgeR-TMM, and DESeq-VS often masked differences among communities when common OTUs differed, and they produced false positives when rare OTUs differed" [2]. On the other hand, for detection of differentially abundant species, "both proportions and rarefied counts result in a high rate of false positives in tests for species that are differentially abundant across sample classes" [3].

In the following examples we will show three popular ways of normalization: relative abundance, rarefaction and cummulative sum scaling.

### 2.2. Normalization by total sum scaling (TSS, relative abundance or proportions) <a id='part2_2'/>

The simplest way to normalize the differences in sample size is to transform the OTU counts table into relative abundance by dividing by the number of total reads of each sample. This type of normalization is also referred to as relative abundance or proportions. We use this normalization to compare taxonomic profiles, while alpha diversity indices are computed on the non-normalized matrix. The reason to do so is that we need a matrix of integer numbers as input.

1) Compute alpha diversity indices and display plots.

In [ ]:
options(repr.plot.width=18, repr.plot.height=6)
plot_richness(ps_final, x = "env_feature", color = "env_feature") + geom_boxplot()

2) Transform taxonomy raw-counts matrix into relative abundance.

In [ ]:
relab_ps = transform_sample_counts(ps_final, function(x) x/sum(x))

3) Agglomerate taxonomy at Class rank and keep only the most abundant classes (threshold=1%, i.e. 0.01). In microbial data, we expect to observe abundance distributions with a long 'tail' of low-abundance organisms which often comprise the large majority of species. For this reason, once the matrix has been transformed to relative abundance, we will show the taxonomic profile at a high taxonomic rank (Class), agglomerating the counts first and using an abundance threshold of 1% to avoid displaying too many unreadable categories in the plot.

In [ ]:
psglom = tax_glom(relab_ps, "Class")
norare_ps = filter_taxa(psglom, function(x) mean(x) > 0.01, TRUE)

4) Visualise the profile in barplots at Class rank in two visualization modes.

In [ ]:
plot_bar(norare_ps, fill = "Class") + facet_wrap(~Class) 
plot_bar(norare_ps, fill = "Class")

### 2.3. Normalization by subsampling (rarefaction) <a id='part2_3'/>

Rarefaction is an alternative to relative abundance normalization to obtain an adjusted OTUs count matrix. The method is based on a process of subsampling to the smallest library size in the data set. The algorithm randomly removes reads until the samples reach the same library size. Despite the apparent disadvantage of discarding information from the larger samples, rarefaction is quite popular in microbial ecology.

The first step is to find the smallest sample size. We can use the number of observed OTUs in the original matrix to do so.

1) Find the smallest sample size.

In [ ]:
head(estimate_richness(ps_final)[order(estimate_richness(ps_final)$Observed),], 1)

2) Rarefying to the smallest sample.

In [ ]:
ps_rare=rarefy_even_depth(ps_final, sample.size=23, replace = FALSE, rngseed=123, verbose = FALSE)
#otu_table(ps_rare)

3) Plot diversity indices.

In [ ]:
plot_richness(ps_rare, x = "env_feature", color = "env_feature") + geom_boxplot()

4) Aglomerate taxonomy at Class rank and visualize the profile (show the top 15 classes only).

In [ ]:
psglom = tax_glom(ps_rare, "Class")
top15 = names(sort(taxa_sums(psglom), decreasing=TRUE)[1:15])
top15_ps = prune_taxa(top15, psglom)

options(repr.plot.width=18, repr.plot.height=9)
plot_bar(top15_ps, fill = "Class") + facet_wrap(~Class)

options(repr.plot.width=18, repr.plot.height=6)
plot_bar(top15_ps, fill = "Class")

### 2.4. Normalization by cumulative sum scaling (CSS) <a id='part2_4'/>

The third normalization method we are going to apply is CSS. To do so, we will use the implementation on the microbiomeMarker library.
Cumulative sum scaling normalization calculates scaling factors as the cumulative sum of gene abundances up to a data-derived threshold. This method is based on the assumption that the count distributions in each sample are equivalent for low abundance genes up to a certain threshold. Only the segment of each sample's count distribution that is relatively invariant across samples is scaled by CSS.

1) Normalizing the OTU counts in the `ps_final` object.

In [ ]:
ps_CSS = normalize(ps_final, method = "CSS")

2) Compute and plot alpha diversity metrics.

In [ ]:
plot_richness(ps_CSS, x = "env_feature", color = "env_feature") + geom_boxplot()

3) Aglomerate taxonomy at Class rank and visualize the profile (show the top 15 classes only).

In [ ]:
psglom = tax_glom(ps_CSS, "Class")
top15 = names(sort(taxa_sums(psglom), decreasing=TRUE)[1:15])
top15_ps = prune_taxa(top15, psglom)

options(repr.plot.width=18, repr.plot.height=9)
plot_bar(top15_ps, fill = "Class") + facet_wrap(~Class)

options(repr.plot.width=18, repr.plot.height=6)
plot_bar(top15_ps, fill = "Class")

## Part 3. Comparative metagenomics at community-level: Beta diversity <a id='part3'/>

According to Pereira *et al.,* (2018)[4], the best normalization method for metagenomic gene abundance (tested in TARA ocean samples) is CSS for large group sizes. For this reason, we will use this method to show beta-diversity.

1) Compute beta diversity using various methods to calculate distance, and perform principle components analysis ploting the first two axes. We will follow the steps described by [4] to create a list of suitable distance methods (including adonis options for the next step), iterate through them, and display a combined plot. For a better visualization we are going to show the 95% confidence region with an ellipse.

In [ ]:
# Generating the methods list
dist_methods = unlist(distanceMethodList)
dist_methods = dist_methods[c(-(1:4),-(20:47))]

# Iterating through the list to save the plot
plist <- vector("list", length(dist_methods))
names(plist) = dist_methods
for( i in dist_methods ){
    # Calculate distance matrix
    iDist <- distance(ps_CSS, method=i)
    # Calculate ordination
    iMDS  <- ordinate(ps_CSS, "MDS", distance=iDist)
    ## Make plot
    # Don't carry over previous plot (if error, p will be blank)
    p <- NULL
    # Create plot, store as temp variable, p
    p <- plot_ordination(ps_CSS, iMDS, color="env_feature")
    # Add title to each plot
    p <- p + ggtitle(paste("MDS using distance method ", i, sep=""))
    # Save the graphic to file.
    plist[[i]] = p
}

# Create a combined plot
df = ldply(plist, function(x) x$data)
names(df)[1] <- "distance"
p = ggplot(df, aes(Axis.1, Axis.2, color=env_feature))
p = p + geom_point(size=3, alpha=0.5)
p = p + facet_wrap(~distance, scales="free")
p = p + ggtitle("MDS on various distance metrics for TARA ocean dataset") + stat_ellipse(level = 0.95, type="norm", geom = "polygon", alpha=0.3, aes(fill=env_feature)) + theme_bw()

options(repr.plot.width=12, repr.plot.height=10)
p

2) Select the best distance metric – the one which best segregates the data by water layer – and determine whether the two groups of samples have different centroids. To do so, we use a permanova implemented in the `vegan` library's `adonis` function. The method calculates the squared deviations of each site to the centroid and then, performs significance tests using F-tests based on sequential sums of squares from permutations of the raw data.

In [ ]:
metadata = as(sample_data(ps_CSS), "data.frame")
css_beta = distance(ps_CSS, method="canberra")

adonis(css_beta ~ env_feature, data = metadata, perm=1e3)

3) Adonis assumes there is homogeneity of dispersion among groups. Let's test this assumption, to check whether the differences detected by adonis are due to variation in dispersion of the data. The strategy is to run a `betadisper` test (also from the `vegan` library) and  evaluate if there's a significant variation in beta dispersion between groups through an anova.

In [ ]:
bd=betadisper(css_beta, metadata$'env_feature')
bd
anova(bd)

With these results, we can now accept that there is a significant difference between groups and it is not an artifact due to heterogeneous dispersion.

### References: <a id='refs'/>

[1] https://doi.org/10.1038/nrmicro3400

[2] https://doi.org/10.1111/2041-210X.13115

[3] https://doi.org/10.1371/journal.pcbi.1003531

[4] https://doi.org/10.1186/s12864-018-4637-6

[5] https://joey711.github.io/phyloseq/distance.html

Documentation and more MGnifyR code and exercises available [on GitHub](https://beadyallen.github.io/MGnifyR/)

Phyloseq tutorials available [on GitHub](https://joey711.github.io/phyloseq/index.html)
